In [1]:
import os
import math
import time

import numpy as np

from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

from keras import optimizers, callbacks
from keras.utils import np_utils
from keras.layers import Input, Dense
from keras.models import Model

import matplotlib.pyplot as plt


Using TensorFlow backend.


In [2]:
plot_pic_folder = "./graph/"
#  create folder if not exists
if not os.path.exists(plot_pic_folder):
    os.makedirs(plot_pic_folder)
num_of_question = 5
for i in range(1,num_of_question+1):
    if not os.path.exists(plot_pic_folder + str(i) + '/'):
        os.makedirs(plot_pic_folder + str(i) + '/')

In [3]:
class BatchTime(callbacks.Callback):
    def __init__(self):
        self.logs=[]
        self.batch_time = []
        self.start_time = []
        self.cur_start_time = 0
        
        
    def on_batch_begin(self, batch, logs={}):
        self.cur_start_time = time.time()
        self.start_time.append(self.cur_start_time)
        return
 
    def on_batch_end(self, batch, logs={}):
        self.batch_time.append(time.time() - self.cur_start_time)
        return

In [4]:
# scale data
def scale(X, X_min, X_max):
    # min-max normalization
    return (X - X_min)/(X_max-np.min(X, axis=0))

In [5]:
def shuffle_data (samples, labels):
    idx = np.arange(samples.shape[0])
    np.random.shuffle(idx)
    #print  (samples.shape, labels.shape)
    samples, labels = samples[idx], labels[idx]
    return samples, labels

In [6]:
# Point out the maximum and minimum point

def annot_max(x,y, ax=None):
    xmax = x[np.argmax(y)]
    ymax = y.max()
    text= "MAX Point x={:.3f}, y={:.3f}".format(xmax, ymax)
    if not ax:
        ax=plt.gca()
    bbox_props = dict(boxstyle="square,pad=0.3", fc="w", ec="k", lw=0.72)
    arrowprops=dict(arrowstyle="->",connectionstyle="angle,angleA=0,angleB=60")
    kw = dict(xycoords='data',textcoords="axes fraction",
              arrowprops=arrowprops, bbox=bbox_props, ha="right", va="top")
    ax.annotate(text, xy=(xmax, ymax), xytext=(0.94,0.96), **kw)

def annot_min(x,y, ax=None):
    xmin = x[np.argmin(y)]
    ymin = y.min()
    text= "MIN Point x={:.3f}, y={:.3f}".format(xmin, ymin)
    if not ax:
        ax=plt.gca()
    bbox_props = dict(boxstyle="square,pad=0.3", fc="w", ec="k", lw=0.72)
    arrowprops=dict(arrowstyle="->",connectionstyle="angle,angleA=0,angleB=120")
    kw = dict(xycoords='data',textcoords="axes fraction",
              arrowprops=arrowprops, bbox=bbox_props, ha="right", va="bottom")
    ax.annotate(text, xy=(xmin, ymin), xytext=(0.94,0.96), **kw)

In [7]:
#read train data
train_input = np.loadtxt('./data/sat_train.txt',delimiter=' ')
trainX, train_Y = train_input[:,:36], train_input[:,-1].astype(int)
trainX_min, trainX_max = np.min(trainX, axis=0), np.max(trainX, axis=0)
trainX = scale(trainX, trainX_min, trainX_max)

train_Y[train_Y == 7] = 6
trainY = np.zeros((train_Y.shape[0], 6))
trainY[np.arange(train_Y.shape[0]), train_Y-1] = 1


#read test data
test_input = np.loadtxt('./data/sat_test.txt',delimiter=' ')
testX, test_Y = test_input[:,:36], test_input[:,-1].astype(int)

# testX_min, testX_max = np.min(testX, axis=0), np.max(testX, axis=0)
testX = scale(testX, trainX_min, trainX_max)

test_Y[test_Y == 7] = 6
testY = np.zeros((test_Y.shape[0], 6))
testY[np.arange(test_Y.shape[0]), test_Y-1] = 1

del test_Y, train_Y, trainX_min, trainX_max
print(trainX.shape, trainY.shape)
print(testX.shape, testY.shape)

(4435, 36) (4435, 6)
(2000, 36) (2000, 6)


In [9]:
# question 1 and question 2:
decay = 1e-6
learning_rate = 0.01
epoch_num = 1000

batch_time_taken = []
total_time_taken = []
batch_sizes = [4,8,16,32,64]
for batch_size in batch_sizes:
    bt = BatchTime()
    inputs = Input(shape=(trainX.shape[1],))
    dense1 = Dense(10, activation='sigmoid')(inputs)
    predictions = Dense(trainY.shape[1], activation='softmax')(dense1)

    # This creates a model that includes
    # the Input layer and three Dense layers
    model = Model(inputs=inputs, outputs=predictions)
    sgd = optimizers.SGD(lr=learning_rate, decay=decay)
    model.compile(optimizer=sgd,
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    start_time = time.time()
    train_his = model.fit(trainX, trainY, epochs=epoch_num, verbose=2, 
                          validation_data=[testX,testY],
                          batch_size = batch_size,
                          callbacks=[bt])
    total_time_taken.append(time.time() - start_time)
    
#     loss, acc = model.evaluate(testX, testY, verbose=2)

    
    # Plot the training error and the test accuracy against number of epochs
    plt.figure()
    annot_min(range(1, 1+epoch_num), np.array(train_his.history['loss']))
    plt.plot(range(1, 1+epoch_num), train_his.history['loss'], label='train_cost')
    plt.xlabel('iterations')
    plt.ylabel('cross-entropy')
    plt.title('training cost')
    plt.savefig(plot_pic_folder + '2/p1a_batchsize' + str(batch_size) + '_cost.png')
    plt.gcf().clear()
    plt.close()

    plt.figure()
    annot_max(range(1, 1+epoch_num), np.array(train_his.history['val_acc']))
    plt.plot(range(1, 1+epoch_num), train_his.history['val_acc'], label='test_accuracy')
    plt.xlabel('iterations')
    plt.ylabel('accuracy')
    plt.title('test accuracy')

    plt.savefig(plot_pic_folder + '2/p1a_batchsize' + str(batch_size) + '_sample_accuracy.png')
    plt.gcf().clear()
    plt.close()
    
    batch_time_taken.append(np.mean(bt.batch_time))

# Plot the time taken
batch_time_taken = np.array(batch_time_taken) * 1000
plt.figure()
plt.plot(batch_sizes, batch_time_taken, label='time_taken')
plt.xlabel('batch_sizes')
plt.xticks(batch_sizes)
plt.ylabel('time_taken(ms)')
plt.title('time for a weight update')
plt.savefig(plot_pic_folder + '2/p1a_batch_time.png')
# plt.show()
plt.gcf().clear()
plt.close()

plt.figure()
plt.plot(batch_sizes, total_time_taken, label='time_taken')
plt.xlabel('batch_sizes')
plt.xticks(batch_sizes)
plt.ylabel('time_taken(s)')
plt.title('total time for training')
plt.savefig(plot_pic_folder + '2/p1a_total_time.png')
# plt.show()
plt.gcf().clear()
plt.close()

Train on 4435 samples, validate on 2000 samples
Epoch 1/1000
3s - loss: 1.6747 - acc: 0.3835 - val_loss: 1.6411 - val_acc: 0.5950
Epoch 2/1000
2s - loss: 1.5622 - acc: 0.5892 - val_loss: 1.5220 - val_acc: 0.5990
Epoch 3/1000
2s - loss: 1.4218 - acc: 0.6879 - val_loss: 1.3651 - val_acc: 0.7215
Epoch 4/1000
2s - loss: 1.2611 - acc: 0.7405 - val_loss: 1.2069 - val_acc: 0.7280
Epoch 5/1000
2s - loss: 1.1125 - acc: 0.7533 - val_loss: 1.0700 - val_acc: 0.7400
Epoch 6/1000
2s - loss: 0.9875 - acc: 0.7592 - val_loss: 0.9610 - val_acc: 0.7435
Epoch 7/1000
2s - loss: 0.8877 - acc: 0.7727 - val_loss: 0.8763 - val_acc: 0.7615
Epoch 8/1000
2s - loss: 0.8102 - acc: 0.7865 - val_loss: 0.8044 - val_acc: 0.7705
Epoch 9/1000
2s - loss: 0.7483 - acc: 0.7968 - val_loss: 0.7520 - val_acc: 0.7765
Epoch 10/1000
2s - loss: 0.6988 - acc: 0.8018 - val_loss: 0.7054 - val_acc: 0.7825
Epoch 11/1000
2s - loss: 0.6586 - acc: 0.8074 - val_loss: 0.6706 - val_acc: 0.7900
Epoch 12/1000
2s - loss: 0.6261 - acc: 0.8115 - 

2s - loss: 0.3685 - acc: 0.8478 - val_loss: 0.3968 - val_acc: 0.8320
Epoch 101/1000
2s - loss: 0.3680 - acc: 0.8474 - val_loss: 0.3987 - val_acc: 0.8330
Epoch 102/1000
2s - loss: 0.3670 - acc: 0.8505 - val_loss: 0.3950 - val_acc: 0.8305
Epoch 103/1000
2s - loss: 0.3666 - acc: 0.8496 - val_loss: 0.4016 - val_acc: 0.8315
Epoch 104/1000
2s - loss: 0.3669 - acc: 0.8476 - val_loss: 0.3952 - val_acc: 0.8295
Epoch 105/1000
2s - loss: 0.3661 - acc: 0.8498 - val_loss: 0.3956 - val_acc: 0.8290
Epoch 106/1000
2s - loss: 0.3656 - acc: 0.8494 - val_loss: 0.3930 - val_acc: 0.8305
Epoch 107/1000
2s - loss: 0.3651 - acc: 0.8489 - val_loss: 0.3948 - val_acc: 0.8320
Epoch 108/1000
2s - loss: 0.3647 - acc: 0.8492 - val_loss: 0.3944 - val_acc: 0.8285
Epoch 109/1000
2s - loss: 0.3639 - acc: 0.8478 - val_loss: 0.3968 - val_acc: 0.8315
Epoch 110/1000
2s - loss: 0.3631 - acc: 0.8492 - val_loss: 0.3938 - val_acc: 0.8300
Epoch 111/1000
2s - loss: 0.3635 - acc: 0.8489 - val_loss: 0.3913 - val_acc: 0.8330
Epoch 1

2s - loss: 0.3340 - acc: 0.8627 - val_loss: 0.3683 - val_acc: 0.8380
Epoch 199/1000
2s - loss: 0.3347 - acc: 0.8625 - val_loss: 0.3679 - val_acc: 0.8375
Epoch 200/1000
2s - loss: 0.3342 - acc: 0.8625 - val_loss: 0.3653 - val_acc: 0.8400
Epoch 201/1000
2s - loss: 0.3339 - acc: 0.8600 - val_loss: 0.3650 - val_acc: 0.8400
Epoch 202/1000
2s - loss: 0.3343 - acc: 0.8609 - val_loss: 0.3656 - val_acc: 0.8395
Epoch 203/1000
2s - loss: 0.3334 - acc: 0.8629 - val_loss: 0.3665 - val_acc: 0.8405
Epoch 204/1000
1s - loss: 0.3336 - acc: 0.8609 - val_loss: 0.3635 - val_acc: 0.8410
Epoch 205/1000
1s - loss: 0.3335 - acc: 0.8627 - val_loss: 0.3646 - val_acc: 0.8415
Epoch 206/1000
2s - loss: 0.3330 - acc: 0.8636 - val_loss: 0.3647 - val_acc: 0.8415
Epoch 207/1000
2s - loss: 0.3326 - acc: 0.8627 - val_loss: 0.3644 - val_acc: 0.8400
Epoch 208/1000
2s - loss: 0.3328 - acc: 0.8643 - val_loss: 0.3637 - val_acc: 0.8400
Epoch 209/1000
2s - loss: 0.3323 - acc: 0.8616 - val_loss: 0.3641 - val_acc: 0.8395
Epoch 2

2s - loss: 0.3178 - acc: 0.8713 - val_loss: 0.3544 - val_acc: 0.8485
Epoch 297/1000
2s - loss: 0.3168 - acc: 0.8717 - val_loss: 0.3507 - val_acc: 0.8500
Epoch 298/1000
2s - loss: 0.3174 - acc: 0.8703 - val_loss: 0.3527 - val_acc: 0.8430
Epoch 299/1000
2s - loss: 0.3177 - acc: 0.8706 - val_loss: 0.3523 - val_acc: 0.8445
Epoch 300/1000
2s - loss: 0.3156 - acc: 0.8715 - val_loss: 0.3549 - val_acc: 0.8425
Epoch 301/1000
2s - loss: 0.3169 - acc: 0.8710 - val_loss: 0.3504 - val_acc: 0.8475
Epoch 302/1000
2s - loss: 0.3172 - acc: 0.8731 - val_loss: 0.3521 - val_acc: 0.8435
Epoch 303/1000
2s - loss: 0.3171 - acc: 0.8719 - val_loss: 0.3544 - val_acc: 0.8430
Epoch 304/1000
2s - loss: 0.3165 - acc: 0.8710 - val_loss: 0.3550 - val_acc: 0.8455
Epoch 305/1000
2s - loss: 0.3165 - acc: 0.8717 - val_loss: 0.3538 - val_acc: 0.8435
Epoch 306/1000
2s - loss: 0.3166 - acc: 0.8710 - val_loss: 0.3508 - val_acc: 0.8510
Epoch 307/1000
2s - loss: 0.3163 - acc: 0.8710 - val_loss: 0.3513 - val_acc: 0.8475
Epoch 3

2s - loss: 0.3071 - acc: 0.8760 - val_loss: 0.3495 - val_acc: 0.8460
Epoch 395/1000
2s - loss: 0.3070 - acc: 0.8787 - val_loss: 0.3497 - val_acc: 0.8465
Epoch 396/1000
2s - loss: 0.3059 - acc: 0.8791 - val_loss: 0.3486 - val_acc: 0.8460
Epoch 397/1000
2s - loss: 0.3067 - acc: 0.8767 - val_loss: 0.3473 - val_acc: 0.8465
Epoch 398/1000
2s - loss: 0.3068 - acc: 0.8758 - val_loss: 0.3487 - val_acc: 0.8500
Epoch 399/1000
2s - loss: 0.3071 - acc: 0.8771 - val_loss: 0.3490 - val_acc: 0.8465
Epoch 400/1000
2s - loss: 0.3068 - acc: 0.8767 - val_loss: 0.3478 - val_acc: 0.8465
Epoch 401/1000
2s - loss: 0.3067 - acc: 0.8762 - val_loss: 0.3480 - val_acc: 0.8485
Epoch 402/1000
2s - loss: 0.3064 - acc: 0.8760 - val_loss: 0.3488 - val_acc: 0.8480
Epoch 403/1000
2s - loss: 0.3067 - acc: 0.8778 - val_loss: 0.3464 - val_acc: 0.8475
Epoch 404/1000
2s - loss: 0.3060 - acc: 0.8794 - val_loss: 0.3445 - val_acc: 0.8520
Epoch 405/1000
2s - loss: 0.3057 - acc: 0.8764 - val_loss: 0.3451 - val_acc: 0.8505
Epoch 4

...

0s - loss: 0.4460 - acc: 0.8345 - val_loss: 0.4715 - val_acc: 0.8145
Epoch 515/1000
0s - loss: 0.4458 - acc: 0.8347 - val_loss: 0.4712 - val_acc: 0.8145
Epoch 516/1000
0s - loss: 0.4455 - acc: 0.8349 - val_loss: 0.4711 - val_acc: 0.8135
Epoch 517/1000
0s - loss: 0.4454 - acc: 0.8338 - val_loss: 0.4708 - val_acc: 0.8150
Epoch 518/1000
0s - loss: 0.4452 - acc: 0.8352 - val_loss: 0.4706 - val_acc: 0.8140
Epoch 519/1000
0s - loss: 0.4451 - acc: 0.8343 - val_loss: 0.4705 - val_acc: 0.8145
Epoch 520/1000
0s - loss: 0.4448 - acc: 0.8347 - val_loss: 0.4703 - val_acc: 0.8145
Epoch 521/1000
0s - loss: 0.4447 - acc: 0.8349 - val_loss: 0.4701 - val_acc: 0.8145
Epoch 522/1000
0s - loss: 0.4445 - acc: 0.8345 - val_loss: 0.4699 - val_acc: 0.8160
Epoch 523/1000
0s - loss: 0.4443 - acc: 0.8356 - val_loss: 0.4698 - val_acc: 0.8150
Epoch 524/1000
0s - loss: 0.4440 - acc: 0.8356 - val_loss: 0.4697 - val_acc: 0.8135
Epoch 525/1000
0s - loss: 0.4440 - acc: 0.8356 - val_loss: 0.4694 - val_acc: 0.8145
Epoch 5

0s - loss: 0.4305 - acc: 0.8381 - val_loss: 0.4559 - val_acc: 0.8160
Epoch 613/1000
0s - loss: 0.4304 - acc: 0.8374 - val_loss: 0.4557 - val_acc: 0.8165
Epoch 614/1000
0s - loss: 0.4303 - acc: 0.8374 - val_loss: 0.4556 - val_acc: 0.8165
Epoch 615/1000
0s - loss: 0.4301 - acc: 0.8377 - val_loss: 0.4555 - val_acc: 0.8150
Epoch 616/1000
0s - loss: 0.4300 - acc: 0.8381 - val_loss: 0.4553 - val_acc: 0.8150
Epoch 617/1000
0s - loss: 0.4299 - acc: 0.8381 - val_loss: 0.4553 - val_acc: 0.8160
Epoch 618/1000
0s - loss: 0.4297 - acc: 0.8381 - val_loss: 0.4552 - val_acc: 0.8160
Epoch 619/1000
0s - loss: 0.4296 - acc: 0.8374 - val_loss: 0.4550 - val_acc: 0.8150
Epoch 620/1000
0s - loss: 0.4295 - acc: 0.8370 - val_loss: 0.4549 - val_acc: 0.8150
Epoch 621/1000
0s - loss: 0.4294 - acc: 0.8381 - val_loss: 0.4547 - val_acc: 0.8165
Epoch 622/1000
0s - loss: 0.4293 - acc: 0.8374 - val_loss: 0.4545 - val_acc: 0.8150
Epoch 623/1000
0s - loss: 0.4291 - acc: 0.8372 - val_loss: 0.4544 - val_acc: 0.8150
Epoch 6

0s - loss: 0.4195 - acc: 0.8408 - val_loss: 0.4448 - val_acc: 0.8190
Epoch 711/1000
0s - loss: 0.4194 - acc: 0.8408 - val_loss: 0.4448 - val_acc: 0.8190
Epoch 712/1000
0s - loss: 0.4193 - acc: 0.8413 - val_loss: 0.4447 - val_acc: 0.8190
Epoch 713/1000
0s - loss: 0.4192 - acc: 0.8408 - val_loss: 0.4445 - val_acc: 0.8190
Epoch 714/1000
0s - loss: 0.4191 - acc: 0.8404 - val_loss: 0.4444 - val_acc: 0.8195
Epoch 715/1000
0s - loss: 0.4190 - acc: 0.8399 - val_loss: 0.4443 - val_acc: 0.8195
Epoch 716/1000
0s - loss: 0.4189 - acc: 0.8406 - val_loss: 0.4442 - val_acc: 0.8190
Epoch 717/1000
0s - loss: 0.4188 - acc: 0.8413 - val_loss: 0.4441 - val_acc: 0.8195
Epoch 718/1000
0s - loss: 0.4187 - acc: 0.8410 - val_loss: 0.4440 - val_acc: 0.8200
Epoch 719/1000
0s - loss: 0.4186 - acc: 0.8406 - val_loss: 0.4439 - val_acc: 0.8200
Epoch 720/1000
0s - loss: 0.4185 - acc: 0.8413 - val_loss: 0.4438 - val_acc: 0.8195
Epoch 721/1000
0s - loss: 0.4184 - acc: 0.8408 - val_loss: 0.4437 - val_acc: 0.8190
Epoch 7

0s - loss: 0.4111 - acc: 0.8419 - val_loss: 0.4363 - val_acc: 0.8200
Epoch 809/1000
0s - loss: 0.4110 - acc: 0.8422 - val_loss: 0.4363 - val_acc: 0.8195
Epoch 810/1000
0s - loss: 0.4109 - acc: 0.8415 - val_loss: 0.4362 - val_acc: 0.8195
Epoch 811/1000
0s - loss: 0.4108 - acc: 0.8424 - val_loss: 0.4362 - val_acc: 0.8195
Epoch 812/1000
0s - loss: 0.4107 - acc: 0.8424 - val_loss: 0.4361 - val_acc: 0.8200
Epoch 813/1000
0s - loss: 0.4106 - acc: 0.8426 - val_loss: 0.4361 - val_acc: 0.8210
Epoch 814/1000
0s - loss: 0.4106 - acc: 0.8424 - val_loss: 0.4359 - val_acc: 0.8200
Epoch 815/1000
0s - loss: 0.4105 - acc: 0.8406 - val_loss: 0.4358 - val_acc: 0.8205
Epoch 816/1000
0s - loss: 0.4104 - acc: 0.8422 - val_loss: 0.4358 - val_acc: 0.8195
Epoch 817/1000
0s - loss: 0.4104 - acc: 0.8422 - val_loss: 0.4357 - val_acc: 0.8200
Epoch 818/1000
0s - loss: 0.4103 - acc: 0.8417 - val_loss: 0.4357 - val_acc: 0.8195
Epoch 819/1000
0s - loss: 0.4102 - acc: 0.8426 - val_loss: 0.4356 - val_acc: 0.8200
Epoch 8

0s - loss: 0.4043 - acc: 0.8422 - val_loss: 0.4297 - val_acc: 0.8240
Epoch 907/1000
0s - loss: 0.4041 - acc: 0.8431 - val_loss: 0.4296 - val_acc: 0.8235
Epoch 908/1000
0s - loss: 0.4041 - acc: 0.8428 - val_loss: 0.4296 - val_acc: 0.8225
Epoch 909/1000
0s - loss: 0.4040 - acc: 0.8433 - val_loss: 0.4295 - val_acc: 0.8235
Epoch 910/1000
0s - loss: 0.4039 - acc: 0.8428 - val_loss: 0.4295 - val_acc: 0.8235
Epoch 911/1000
0s - loss: 0.4039 - acc: 0.8426 - val_loss: 0.4295 - val_acc: 0.8230
Epoch 912/1000
0s - loss: 0.4038 - acc: 0.8424 - val_loss: 0.4293 - val_acc: 0.8240
Epoch 913/1000
0s - loss: 0.4037 - acc: 0.8431 - val_loss: 0.4293 - val_acc: 0.8245
Epoch 914/1000
0s - loss: 0.4037 - acc: 0.8424 - val_loss: 0.4292 - val_acc: 0.8250
Epoch 915/1000
0s - loss: 0.4037 - acc: 0.8426 - val_loss: 0.4292 - val_acc: 0.8245
Epoch 916/1000
0s - loss: 0.4036 - acc: 0.8431 - val_loss: 0.4290 - val_acc: 0.8235
Epoch 917/1000
0s - loss: 0.4036 - acc: 0.8426 - val_loss: 0.4290 - val_acc: 0.8235
Epoch 9

In [10]:
#question 3

decay = 1e-6
learning_rate = 0.01
epoch_num = 1000

batch_time_taken = []
total_time_taken = []
neuron_nums = [5,10,15,20,25]
for neuron_num in neuron_nums:
    bt = BatchTime()

    inputs = Input(shape=(trainX.shape[1],))
    dense1 = Dense(neuron_num, activation='sigmoid')(inputs)
    predictions = Dense(trainY.shape[1], activation='softmax')(dense1)

    model = Model(inputs=inputs, outputs=predictions)
    sgd = optimizers.SGD(lr=learning_rate, decay=decay)
    model.compile(optimizer=sgd,
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    start_time = time.time()
    train_his = model.fit(trainX, trainY, epochs=epoch_num, verbose=2, 
                          validation_data=[testX,testY],
                          callbacks=[bt],
                          batch_size = 4)
    total_time_taken.append(time.time() - start_time)
#     loss, acc = model.evaluate(testX, testY, verbose=2)

    
    # Plot the training error and the test accuracy against number of epochs
    plt.figure()
    plt.plot(range(1, 1+epoch_num), train_his.history['loss'], label='train_cost')
    plt.xlabel('iterations')
    plt.ylabel('cross-entropy')
    plt.title('training cost')
    annot_min(range(1, 1+epoch_num), np.array(train_his.history['loss']))
    plt.savefig(plot_pic_folder + '3/p1a_neuron' + str(neuron_num) + '_cost.png')
    plt.gcf().clear()
    plt.close()
    

    plt.figure()
    plt.plot(range(1, 1+epoch_num), train_his.history['val_acc'], label='test_accuracy')
    plt.xlabel('iterations')
    plt.ylabel('accuracy')
    plt.title('test accuracy')
    annot_max(range(1, 1+epoch_num), np.array(train_his.history['val_acc']))
    plt.savefig(plot_pic_folder + '3/p1a_neuron' + str(neuron_num) + '_sample_accuracy.png')
    plt.close()
    plt.gcf().clear()
    plt.close()
    
    batch_time_taken.append(np.mean(bt.batch_time))

# Plot the time taken
batch_time_taken = np.array(batch_time_taken) * 1000
plt.figure()
plt.plot(neuron_nums, batch_time_taken, label='time_taken')
plt.xlabel('neuron_nums')
plt.xticks(neuron_nums)
plt.ylabel('time_taken(ms)')
plt.title('time for a weight update')
plt.savefig(plot_pic_folder + '3/p1a_batch_time.png')
# plt.show()
plt.gcf().clear()
plt.close()

plt.figure()
plt.plot(neuron_nums, total_time_taken, label='time_taken')
plt.xlabel('neuron_nums')
plt.xticks(neuron_nums)
plt.ylabel('time_taken(ms)')
plt.title('total time for training')
plt.savefig(plot_pic_folder + '3/p1a_total_time.png')
# plt.show()
plt.gcf().clear()
plt.close()

Train on 4435 samples, validate on 2000 samples
Epoch 1/1000
3s - loss: 1.6775 - acc: 0.4088 - val_loss: 1.6583 - val_acc: 0.5225
Epoch 2/1000
2s - loss: 1.6063 - acc: 0.4746 - val_loss: 1.5952 - val_acc: 0.4135
Epoch 3/1000
2s - loss: 1.5294 - acc: 0.5617 - val_loss: 1.5037 - val_acc: 0.5970
Epoch 4/1000
2s - loss: 1.4283 - acc: 0.6322 - val_loss: 1.3936 - val_acc: 0.6325
Epoch 5/1000
2s - loss: 1.3104 - acc: 0.7015 - val_loss: 1.2723 - val_acc: 0.7275
Epoch 6/1000
2s - loss: 1.1893 - acc: 0.7506 - val_loss: 1.1551 - val_acc: 0.7395
Epoch 7/1000
2s - loss: 1.0773 - acc: 0.7614 - val_loss: 1.0517 - val_acc: 0.7460
Epoch 8/1000
2s - loss: 0.9813 - acc: 0.7689 - val_loss: 0.9675 - val_acc: 0.7560
Epoch 9/1000
1s - loss: 0.9028 - acc: 0.7768 - val_loss: 0.8962 - val_acc: 0.7670
Epoch 10/1000
2s - loss: 0.8388 - acc: 0.7894 - val_loss: 0.8415 - val_acc: 0.7615
Epoch 11/1000
2s - loss: 0.7867 - acc: 0.7944 - val_loss: 0.7927 - val_acc: 0.7720
Epoch 12/1000
1s - loss: 0.7430 - acc: 0.8007 - 

2s - loss: 0.3810 - acc: 0.8539 - val_loss: 0.4111 - val_acc: 0.8295
Epoch 101/1000
2s - loss: 0.3808 - acc: 0.8523 - val_loss: 0.4132 - val_acc: 0.8250
Epoch 102/1000
2s - loss: 0.3804 - acc: 0.8514 - val_loss: 0.4126 - val_acc: 0.8250
Epoch 103/1000
2s - loss: 0.3803 - acc: 0.8480 - val_loss: 0.4109 - val_acc: 0.8270
Epoch 104/1000
2s - loss: 0.3800 - acc: 0.8485 - val_loss: 0.4108 - val_acc: 0.8305
Epoch 105/1000
2s - loss: 0.3790 - acc: 0.8489 - val_loss: 0.4116 - val_acc: 0.8300
Epoch 106/1000
2s - loss: 0.3791 - acc: 0.8516 - val_loss: 0.4089 - val_acc: 0.8270
Epoch 107/1000
2s - loss: 0.3787 - acc: 0.8510 - val_loss: 0.4080 - val_acc: 0.8280
Epoch 108/1000
2s - loss: 0.3781 - acc: 0.8487 - val_loss: 0.4089 - val_acc: 0.8300
Epoch 109/1000
2s - loss: 0.3774 - acc: 0.8507 - val_loss: 0.4117 - val_acc: 0.8280
Epoch 110/1000
2s - loss: 0.3776 - acc: 0.8519 - val_loss: 0.4094 - val_acc: 0.8285
Epoch 111/1000
2s - loss: 0.3772 - acc: 0.8521 - val_loss: 0.4094 - val_acc: 0.8320
Epoch 1

2s - loss: 0.3545 - acc: 0.8570 - val_loss: 0.3874 - val_acc: 0.8315
Epoch 199/1000
2s - loss: 0.3541 - acc: 0.8570 - val_loss: 0.3889 - val_acc: 0.8295
Epoch 200/1000
2s - loss: 0.3544 - acc: 0.8579 - val_loss: 0.3864 - val_acc: 0.8340
Epoch 201/1000
2s - loss: 0.3539 - acc: 0.8582 - val_loss: 0.3865 - val_acc: 0.8335
Epoch 202/1000
2s - loss: 0.3539 - acc: 0.8573 - val_loss: 0.3896 - val_acc: 0.8310
Epoch 203/1000
2s - loss: 0.3538 - acc: 0.8570 - val_loss: 0.3863 - val_acc: 0.8310
Epoch 204/1000
2s - loss: 0.3535 - acc: 0.8568 - val_loss: 0.3859 - val_acc: 0.8345
Epoch 205/1000
2s - loss: 0.3535 - acc: 0.8600 - val_loss: 0.3857 - val_acc: 0.8320
Epoch 206/1000
2s - loss: 0.3530 - acc: 0.8589 - val_loss: 0.3852 - val_acc: 0.8340
Epoch 207/1000
2s - loss: 0.3529 - acc: 0.8584 - val_loss: 0.3845 - val_acc: 0.8355
Epoch 208/1000
2s - loss: 0.3523 - acc: 0.8598 - val_loss: 0.3854 - val_acc: 0.8380
Epoch 209/1000
2s - loss: 0.3524 - acc: 0.8575 - val_loss: 0.3855 - val_acc: 0.8300
Epoch 2

2s - loss: 0.3411 - acc: 0.8595 - val_loss: 0.3742 - val_acc: 0.8365
Epoch 297/1000
2s - loss: 0.3399 - acc: 0.8616 - val_loss: 0.3753 - val_acc: 0.8330
Epoch 298/1000
2s - loss: 0.3408 - acc: 0.8620 - val_loss: 0.3742 - val_acc: 0.8340
Epoch 299/1000
2s - loss: 0.3400 - acc: 0.8620 - val_loss: 0.3758 - val_acc: 0.8330
Epoch 300/1000
2s - loss: 0.3401 - acc: 0.8598 - val_loss: 0.3721 - val_acc: 0.8380
Epoch 301/1000
2s - loss: 0.3401 - acc: 0.8629 - val_loss: 0.3720 - val_acc: 0.8390
Epoch 302/1000
2s - loss: 0.3402 - acc: 0.8613 - val_loss: 0.3760 - val_acc: 0.8345
Epoch 303/1000
2s - loss: 0.3399 - acc: 0.8613 - val_loss: 0.3749 - val_acc: 0.8340
Epoch 304/1000
2s - loss: 0.3400 - acc: 0.8613 - val_loss: 0.3716 - val_acc: 0.8395
Epoch 305/1000
2s - loss: 0.3398 - acc: 0.8629 - val_loss: 0.3719 - val_acc: 0.8390
Epoch 306/1000
2s - loss: 0.3394 - acc: 0.8604 - val_loss: 0.3767 - val_acc: 0.8335
Epoch 307/1000
2s - loss: 0.3396 - acc: 0.8607 - val_loss: 0.3789 - val_acc: 0.8335
Epoch 3

2s - loss: 0.3315 - acc: 0.8649 - val_loss: 0.3661 - val_acc: 0.8395
Epoch 395/1000
2s - loss: 0.3311 - acc: 0.8658 - val_loss: 0.3653 - val_acc: 0.8410
Epoch 396/1000
2s - loss: 0.3311 - acc: 0.8640 - val_loss: 0.3662 - val_acc: 0.8385
Epoch 397/1000
2s - loss: 0.3309 - acc: 0.8634 - val_loss: 0.3662 - val_acc: 0.8380
Epoch 398/1000
2s - loss: 0.3305 - acc: 0.8652 - val_loss: 0.3658 - val_acc: 0.8405
Epoch 399/1000
2s - loss: 0.3303 - acc: 0.8643 - val_loss: 0.3670 - val_acc: 0.8345
Epoch 400/1000
2s - loss: 0.3302 - acc: 0.8643 - val_loss: 0.3649 - val_acc: 0.8410
Epoch 401/1000
2s - loss: 0.3302 - acc: 0.8636 - val_loss: 0.3671 - val_acc: 0.8365
Epoch 402/1000
2s - loss: 0.3303 - acc: 0.8658 - val_loss: 0.3666 - val_acc: 0.8375
Epoch 403/1000
2s - loss: 0.3300 - acc: 0.8652 - val_loss: 0.3680 - val_acc: 0.8350
Epoch 404/1000
2s - loss: 0.3295 - acc: 0.8645 - val_loss: 0.3751 - val_acc: 0.8320
Epoch 405/1000
2s - loss: 0.3302 - acc: 0.8634 - val_loss: 0.3634 - val_acc: 0.8425
Epoch 4

...

2s - loss: 0.2953 - acc: 0.8841 - val_loss: 0.3350 - val_acc: 0.8520
Epoch 514/1000
2s - loss: 0.2952 - acc: 0.8846 - val_loss: 0.3371 - val_acc: 0.8540
Epoch 515/1000
2s - loss: 0.2950 - acc: 0.8843 - val_loss: 0.3334 - val_acc: 0.8565
Epoch 516/1000
2s - loss: 0.2953 - acc: 0.8816 - val_loss: 0.3328 - val_acc: 0.8605
Epoch 517/1000
2s - loss: 0.2954 - acc: 0.8839 - val_loss: 0.3331 - val_acc: 0.8570
Epoch 518/1000
2s - loss: 0.2948 - acc: 0.8837 - val_loss: 0.3351 - val_acc: 0.8590
Epoch 519/1000
2s - loss: 0.2939 - acc: 0.8839 - val_loss: 0.3361 - val_acc: 0.8545
Epoch 520/1000
2s - loss: 0.2947 - acc: 0.8852 - val_loss: 0.3332 - val_acc: 0.8540
Epoch 521/1000
2s - loss: 0.2946 - acc: 0.8846 - val_loss: 0.3353 - val_acc: 0.8530
Epoch 522/1000
2s - loss: 0.2947 - acc: 0.8839 - val_loss: 0.3329 - val_acc: 0.8570
Epoch 523/1000
2s - loss: 0.2941 - acc: 0.8837 - val_loss: 0.3348 - val_acc: 0.8560
Epoch 524/1000
2s - loss: 0.2947 - acc: 0.8821 - val_loss: 0.3320 - val_acc: 0.8590
Epoch 5

2s - loss: 0.2877 - acc: 0.8877 - val_loss: 0.3300 - val_acc: 0.8595
Epoch 612/1000
2s - loss: 0.2877 - acc: 0.8893 - val_loss: 0.3300 - val_acc: 0.8615
Epoch 613/1000
2s - loss: 0.2875 - acc: 0.8888 - val_loss: 0.3325 - val_acc: 0.8530
Epoch 614/1000
2s - loss: 0.2879 - acc: 0.8873 - val_loss: 0.3295 - val_acc: 0.8620
Epoch 615/1000
2s - loss: 0.2877 - acc: 0.8861 - val_loss: 0.3303 - val_acc: 0.8605
Epoch 616/1000
2s - loss: 0.2872 - acc: 0.8857 - val_loss: 0.3312 - val_acc: 0.8565
Epoch 617/1000
2s - loss: 0.2872 - acc: 0.8877 - val_loss: 0.3291 - val_acc: 0.8590
Epoch 618/1000
2s - loss: 0.2870 - acc: 0.8902 - val_loss: 0.3289 - val_acc: 0.8610
Epoch 619/1000
2s - loss: 0.2871 - acc: 0.8891 - val_loss: 0.3292 - val_acc: 0.8625
Epoch 620/1000
2s - loss: 0.2875 - acc: 0.8873 - val_loss: 0.3298 - val_acc: 0.8625
Epoch 621/1000
2s - loss: 0.2873 - acc: 0.8879 - val_loss: 0.3314 - val_acc: 0.8610
Epoch 622/1000
2s - loss: 0.2872 - acc: 0.8886 - val_loss: 0.3339 - val_acc: 0.8560
Epoch 6

2s - loss: 0.2814 - acc: 0.8915 - val_loss: 0.3324 - val_acc: 0.8575
Epoch 710/1000
2s - loss: 0.2812 - acc: 0.8902 - val_loss: 0.3310 - val_acc: 0.8530
Epoch 711/1000
2s - loss: 0.2815 - acc: 0.8884 - val_loss: 0.3335 - val_acc: 0.8560
Epoch 712/1000
2s - loss: 0.2812 - acc: 0.8906 - val_loss: 0.3344 - val_acc: 0.8635
Epoch 713/1000
2s - loss: 0.2818 - acc: 0.8931 - val_loss: 0.3282 - val_acc: 0.8600
Epoch 714/1000
2s - loss: 0.2814 - acc: 0.8924 - val_loss: 0.3285 - val_acc: 0.8575
Epoch 715/1000
2s - loss: 0.2815 - acc: 0.8918 - val_loss: 0.3273 - val_acc: 0.8590
Epoch 716/1000
2s - loss: 0.2814 - acc: 0.8924 - val_loss: 0.3267 - val_acc: 0.8625
Epoch 717/1000
2s - loss: 0.2810 - acc: 0.8915 - val_loss: 0.3270 - val_acc: 0.8580
Epoch 718/1000
2s - loss: 0.2817 - acc: 0.8897 - val_loss: 0.3263 - val_acc: 0.8605
Epoch 719/1000
2s - loss: 0.2808 - acc: 0.8918 - val_loss: 0.3253 - val_acc: 0.8645
Epoch 720/1000
2s - loss: 0.2803 - acc: 0.8920 - val_loss: 0.3280 - val_acc: 0.8575
Epoch 7

2s - loss: 0.2759 - acc: 0.8929 - val_loss: 0.3243 - val_acc: 0.8635
Epoch 808/1000
2s - loss: 0.2763 - acc: 0.8924 - val_loss: 0.3283 - val_acc: 0.8580
Epoch 809/1000
2s - loss: 0.2762 - acc: 0.8954 - val_loss: 0.3243 - val_acc: 0.8650
Epoch 810/1000
2s - loss: 0.2759 - acc: 0.8945 - val_loss: 0.3230 - val_acc: 0.8620
Epoch 811/1000
2s - loss: 0.2754 - acc: 0.8933 - val_loss: 0.3280 - val_acc: 0.8675
Epoch 812/1000
2s - loss: 0.2759 - acc: 0.8931 - val_loss: 0.3260 - val_acc: 0.8655
Epoch 813/1000
2s - loss: 0.2764 - acc: 0.8940 - val_loss: 0.3221 - val_acc: 0.8675
Epoch 814/1000
2s - loss: 0.2758 - acc: 0.8945 - val_loss: 0.3241 - val_acc: 0.8630
Epoch 815/1000
2s - loss: 0.2761 - acc: 0.8918 - val_loss: 0.3276 - val_acc: 0.8655
Epoch 816/1000
2s - loss: 0.2753 - acc: 0.8938 - val_loss: 0.3260 - val_acc: 0.8660
Epoch 817/1000
2s - loss: 0.2760 - acc: 0.8936 - val_loss: 0.3220 - val_acc: 0.8665
Epoch 818/1000
2s - loss: 0.2754 - acc: 0.8938 - val_loss: 0.3249 - val_acc: 0.8695
Epoch 8

2s - loss: 0.2704 - acc: 0.8961 - val_loss: 0.3204 - val_acc: 0.8635
Epoch 906/1000
2s - loss: 0.2708 - acc: 0.8929 - val_loss: 0.3282 - val_acc: 0.8570
Epoch 907/1000
2s - loss: 0.2708 - acc: 0.8965 - val_loss: 0.3239 - val_acc: 0.8660
Epoch 908/1000
2s - loss: 0.2712 - acc: 0.8945 - val_loss: 0.3265 - val_acc: 0.8630
Epoch 909/1000
2s - loss: 0.2709 - acc: 0.8927 - val_loss: 0.3263 - val_acc: 0.8655
Epoch 910/1000
2s - loss: 0.2707 - acc: 0.8954 - val_loss: 0.3218 - val_acc: 0.8710
Epoch 911/1000
2s - loss: 0.2705 - acc: 0.8945 - val_loss: 0.3263 - val_acc: 0.8570
Epoch 912/1000
2s - loss: 0.2710 - acc: 0.8927 - val_loss: 0.3231 - val_acc: 0.8635
Epoch 913/1000
2s - loss: 0.2705 - acc: 0.8933 - val_loss: 0.3249 - val_acc: 0.8635
Epoch 914/1000
2s - loss: 0.2707 - acc: 0.8970 - val_loss: 0.3190 - val_acc: 0.8710
Epoch 915/1000
2s - loss: 0.2699 - acc: 0.8958 - val_loss: 0.3209 - val_acc: 0.8675
Epoch 916/1000
2s - loss: 0.2705 - acc: 0.8954 - val_loss: 0.3195 - val_acc: 0.8680
Epoch 9

In [11]:
##### question 4

learning_rate = 0.01
epoch_num = 1000

batch_time_taken = []
total_time_taken = []

decay_nums = [0,math.pow(10,-3),math.pow(10,-6),math.pow(10,-9),math.pow(10,-9)]
test_accuracy = []
for decay in decay_nums:
    bt = BatchTime()
    start_time = time.time()
    inputs = Input(shape=(trainX.shape[1],))
    dense1 = Dense(25, activation='sigmoid')(inputs)
    predictions = Dense(trainY.shape[1], activation='softmax')(dense1)

    model = Model(inputs=inputs, outputs=predictions)
    sgd = optimizers.SGD(lr=learning_rate, decay=decay)
    model.compile(optimizer=sgd,
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    start_time = time.time()
    train_his = model.fit(trainX, trainY, epochs=epoch_num, verbose=2, 
                          validation_data=[testX,testY],
                          callbacks=[bt],
                          batch_size = 4)

    total_time_taken.append(time.time() - start_time)
#     loss, acc = model.evaluate(testX, testY, verbose=2)

    
    # Plot the training error against number of epochs
    plt.figure()
    plt.plot(range(1, 1+epoch_num), train_his.history['loss'], label='train_cost')
    plt.xlabel('iterations')
    plt.ylabel('cross-entropy')
    plt.title('training cost')
    annot_min(range(1, 1+epoch_num), np.array(train_his.history['loss']))
    plt.savefig(plot_pic_folder + '4/p1a_decay' + str(decay) + '_cost.png')
    plt.gcf().clear()
    plt.close()
    
    test_accuracy.append(max(train_his.history['val_acc']))
    batch_time_taken.append(np.mean(bt.batch_time))
    
#Plot the test accuracy against the different values of decay parameter.
plt.figure()
plt.plot(decay_nums, test_accuracy, label='test accuracy')
plt.xlabel('decay_nums')
plt.ylabel('accuracy')
plt.title('test accuracy')
annot_max(decay_nums, np.array(test_accuracy))
plt.savefig(plot_pic_folder + '4/p1a_test_accuracy.png')
plt.close()
plt.gcf().clear()
plt.close()
    

# Plot the time taken
batch_time_taken = np.array(batch_time_taken) * 1000
plt.figure()
plt.plot(decay_nums, batch_time_taken, label='time_taken')
plt.xlabel('decay_nums')
plt.xticks(decay_nums)
plt.ylabel('time_taken(ms)')
plt.title('time for a weight update')
plt.savefig(plot_pic_folder + '4/p1a_batch_time.png')
# plt.show()
plt.gcf().clear()
plt.close()

plt.figure()
plt.plot(decay_nums, total_time_taken, label='time_taken')
plt.xlabel('decay_nums')
plt.xticks(decay_nums)
plt.ylabel('time_taken(ms)')
plt.title('total time for training')
plt.savefig(plot_pic_folder + '4/p1a_total_time.png')
# plt.show()
plt.gcf().clear()
plt.close()

Train on 4435 samples, validate on 2000 samples
Epoch 1/1000
2s - loss: 1.6039 - acc: 0.4809 - val_loss: 1.5441 - val_acc: 0.5890
Epoch 2/1000
2s - loss: 1.4109 - acc: 0.6794 - val_loss: 1.3320 - val_acc: 0.6975
Epoch 3/1000
1s - loss: 1.2101 - acc: 0.7400 - val_loss: 1.1442 - val_acc: 0.7070
Epoch 4/1000
2s - loss: 1.0367 - acc: 0.7504 - val_loss: 0.9909 - val_acc: 0.7420
Epoch 5/1000
1s - loss: 0.9028 - acc: 0.7603 - val_loss: 0.8813 - val_acc: 0.7410
Epoch 6/1000
1s - loss: 0.8040 - acc: 0.7745 - val_loss: 0.7939 - val_acc: 0.7590
Epoch 7/1000
2s - loss: 0.7329 - acc: 0.7876 - val_loss: 0.7340 - val_acc: 0.7625
Epoch 8/1000
2s - loss: 0.6789 - acc: 0.7991 - val_loss: 0.6842 - val_acc: 0.7810
Epoch 9/1000
1s - loss: 0.6360 - acc: 0.8047 - val_loss: 0.6508 - val_acc: 0.7890
Epoch 10/1000
1s - loss: 0.6031 - acc: 0.8101 - val_loss: 0.6191 - val_acc: 0.7905
Epoch 11/1000
1s - loss: 0.5773 - acc: 0.8138 - val_loss: 0.5949 - val_acc: 0.7960
Epoch 12/1000
1s - loss: 0.5556 - acc: 0.8169 - 

2s - loss: 0.3670 - acc: 0.8469 - val_loss: 0.3939 - val_acc: 0.8295
Epoch 101/1000
1s - loss: 0.3669 - acc: 0.8512 - val_loss: 0.3932 - val_acc: 0.8310
Epoch 102/1000
2s - loss: 0.3676 - acc: 0.8516 - val_loss: 0.3923 - val_acc: 0.8280
Epoch 103/1000
2s - loss: 0.3665 - acc: 0.8492 - val_loss: 0.3972 - val_acc: 0.8275
Epoch 104/1000
2s - loss: 0.3653 - acc: 0.8498 - val_loss: 0.3949 - val_acc: 0.8300
Epoch 105/1000
1s - loss: 0.3656 - acc: 0.8512 - val_loss: 0.3929 - val_acc: 0.8285
Epoch 106/1000
2s - loss: 0.3645 - acc: 0.8516 - val_loss: 0.3951 - val_acc: 0.8240
Epoch 107/1000
2s - loss: 0.3642 - acc: 0.8514 - val_loss: 0.3913 - val_acc: 0.8315
Epoch 108/1000
2s - loss: 0.3637 - acc: 0.8498 - val_loss: 0.3962 - val_acc: 0.8255
Epoch 109/1000
2s - loss: 0.3630 - acc: 0.8543 - val_loss: 0.3885 - val_acc: 0.8315
Epoch 110/1000
2s - loss: 0.3636 - acc: 0.8505 - val_loss: 0.3904 - val_acc: 0.8300
Epoch 111/1000
2s - loss: 0.3637 - acc: 0.8503 - val_loss: 0.3893 - val_acc: 0.8280
Epoch 1

2s - loss: 0.3356 - acc: 0.8654 - val_loss: 0.3645 - val_acc: 0.8365
Epoch 199/1000
2s - loss: 0.3352 - acc: 0.8640 - val_loss: 0.3655 - val_acc: 0.8395
Epoch 200/1000
2s - loss: 0.3350 - acc: 0.8640 - val_loss: 0.3643 - val_acc: 0.8415
Epoch 201/1000
2s - loss: 0.3350 - acc: 0.8647 - val_loss: 0.3653 - val_acc: 0.8405
Epoch 202/1000
2s - loss: 0.3347 - acc: 0.8652 - val_loss: 0.3756 - val_acc: 0.8350
Epoch 203/1000
2s - loss: 0.3348 - acc: 0.8649 - val_loss: 0.3630 - val_acc: 0.8415
Epoch 204/1000
2s - loss: 0.3345 - acc: 0.8629 - val_loss: 0.3660 - val_acc: 0.8360
Epoch 205/1000
2s - loss: 0.3340 - acc: 0.8649 - val_loss: 0.3628 - val_acc: 0.8385
Epoch 206/1000
3s - loss: 0.3345 - acc: 0.8667 - val_loss: 0.3616 - val_acc: 0.8445
Epoch 207/1000
4s - loss: 0.3345 - acc: 0.8667 - val_loss: 0.3626 - val_acc: 0.8420
Epoch 208/1000
3s - loss: 0.3332 - acc: 0.8645 - val_loss: 0.3663 - val_acc: 0.8390
Epoch 209/1000
2s - loss: 0.3337 - acc: 0.8665 - val_loss: 0.3657 - val_acc: 0.8355
Epoch 2

3s - loss: 0.3149 - acc: 0.8760 - val_loss: 0.3489 - val_acc: 0.8520
Epoch 297/1000
2s - loss: 0.3156 - acc: 0.8746 - val_loss: 0.3458 - val_acc: 0.8505
Epoch 298/1000
2s - loss: 0.3144 - acc: 0.8733 - val_loss: 0.3456 - val_acc: 0.8565
Epoch 299/1000
2s - loss: 0.3140 - acc: 0.8728 - val_loss: 0.3511 - val_acc: 0.8420
Epoch 300/1000
2s - loss: 0.3143 - acc: 0.8742 - val_loss: 0.3477 - val_acc: 0.8465
Epoch 301/1000
2s - loss: 0.3137 - acc: 0.8758 - val_loss: 0.3506 - val_acc: 0.8465
Epoch 302/1000
2s - loss: 0.3134 - acc: 0.8740 - val_loss: 0.3495 - val_acc: 0.8500
Epoch 303/1000
2s - loss: 0.3126 - acc: 0.8780 - val_loss: 0.3546 - val_acc: 0.8580
Epoch 304/1000
2s - loss: 0.3122 - acc: 0.8733 - val_loss: 0.3486 - val_acc: 0.8545
Epoch 305/1000
2s - loss: 0.3144 - acc: 0.8731 - val_loss: 0.3460 - val_acc: 0.8480
Epoch 306/1000
2s - loss: 0.3134 - acc: 0.8758 - val_loss: 0.3474 - val_acc: 0.8480
Epoch 307/1000
2s - loss: 0.3125 - acc: 0.8753 - val_loss: 0.3525 - val_acc: 0.8435
Epoch 3

1s - loss: 0.2988 - acc: 0.8798 - val_loss: 0.3364 - val_acc: 0.8525
Epoch 395/1000
1s - loss: 0.2991 - acc: 0.8812 - val_loss: 0.3364 - val_acc: 0.8555
Epoch 396/1000
1s - loss: 0.2983 - acc: 0.8850 - val_loss: 0.3354 - val_acc: 0.8555
Epoch 397/1000
1s - loss: 0.2982 - acc: 0.8821 - val_loss: 0.3415 - val_acc: 0.8515
Epoch 398/1000
1s - loss: 0.2989 - acc: 0.8823 - val_loss: 0.3352 - val_acc: 0.8660
Epoch 399/1000
1s - loss: 0.2982 - acc: 0.8828 - val_loss: 0.3351 - val_acc: 0.8570
Epoch 400/1000
1s - loss: 0.2983 - acc: 0.8823 - val_loss: 0.3374 - val_acc: 0.8650
Epoch 401/1000
1s - loss: 0.2989 - acc: 0.8805 - val_loss: 0.3360 - val_acc: 0.8575
Epoch 402/1000
1s - loss: 0.2985 - acc: 0.8823 - val_loss: 0.3404 - val_acc: 0.8495
Epoch 403/1000
1s - loss: 0.2979 - acc: 0.8800 - val_loss: 0.3366 - val_acc: 0.8535
Epoch 404/1000
1s - loss: 0.2972 - acc: 0.8816 - val_loss: 0.3371 - val_acc: 0.8540
Epoch 405/1000
1s - loss: 0.2971 - acc: 0.8837 - val_loss: 0.3386 - val_acc: 0.8565
Epoch 4

...

2s - loss: 0.2727 - acc: 0.8938 - val_loss: 0.3261 - val_acc: 0.8590
Epoch 631/1000
2s - loss: 0.2724 - acc: 0.8931 - val_loss: 0.3215 - val_acc: 0.8665
Epoch 632/1000
2s - loss: 0.2722 - acc: 0.8943 - val_loss: 0.3205 - val_acc: 0.8660
Epoch 633/1000
2s - loss: 0.2728 - acc: 0.8947 - val_loss: 0.3165 - val_acc: 0.8730
Epoch 634/1000
2s - loss: 0.2717 - acc: 0.8929 - val_loss: 0.3204 - val_acc: 0.8670
Epoch 635/1000
1s - loss: 0.2714 - acc: 0.8958 - val_loss: 0.3278 - val_acc: 0.8640
Epoch 636/1000
2s - loss: 0.2721 - acc: 0.8922 - val_loss: 0.3203 - val_acc: 0.8645
Epoch 637/1000
2s - loss: 0.2717 - acc: 0.8920 - val_loss: 0.3163 - val_acc: 0.8675
Epoch 638/1000
2s - loss: 0.2715 - acc: 0.8929 - val_loss: 0.3183 - val_acc: 0.8625
Epoch 639/1000
2s - loss: 0.2716 - acc: 0.8947 - val_loss: 0.3171 - val_acc: 0.8720
Epoch 640/1000
2s - loss: 0.2713 - acc: 0.8947 - val_loss: 0.3195 - val_acc: 0.8725
Epoch 641/1000
1s - loss: 0.2721 - acc: 0.8938 - val_loss: 0.3168 - val_acc: 0.8745
Epoch 6

2s - loss: 0.2623 - acc: 0.8985 - val_loss: 0.3126 - val_acc: 0.8655
Epoch 729/1000
4s - loss: 0.2620 - acc: 0.8970 - val_loss: 0.3112 - val_acc: 0.8685
Epoch 730/1000
5s - loss: 0.2622 - acc: 0.8972 - val_loss: 0.3103 - val_acc: 0.8750
Epoch 731/1000
235s - loss: 0.2612 - acc: 0.8994 - val_loss: 0.3129 - val_acc: 0.8630
Epoch 732/1000
2s - loss: 0.2618 - acc: 0.8983 - val_loss: 0.3133 - val_acc: 0.8620
Epoch 733/1000
2s - loss: 0.2619 - acc: 0.8979 - val_loss: 0.3178 - val_acc: 0.8575
Epoch 734/1000
2s - loss: 0.2605 - acc: 0.8999 - val_loss: 0.3109 - val_acc: 0.8765
Epoch 735/1000
2s - loss: 0.2611 - acc: 0.8990 - val_loss: 0.3104 - val_acc: 0.8775
Epoch 736/1000
4s - loss: 0.2616 - acc: 0.8983 - val_loss: 0.3133 - val_acc: 0.8665
Epoch 737/1000
4s - loss: 0.2609 - acc: 0.8994 - val_loss: 0.3114 - val_acc: 0.8760
Epoch 738/1000
4s - loss: 0.2613 - acc: 0.8976 - val_loss: 0.3091 - val_acc: 0.8765
Epoch 739/1000
4s - loss: 0.2615 - acc: 0.8994 - val_loss: 0.3126 - val_acc: 0.8690
Epoch

1s - loss: 0.2530 - acc: 0.9024 - val_loss: 0.3013 - val_acc: 0.8805
Epoch 827/1000
2s - loss: 0.2535 - acc: 0.9017 - val_loss: 0.3028 - val_acc: 0.8770
Epoch 828/1000
1s - loss: 0.2532 - acc: 0.8994 - val_loss: 0.3036 - val_acc: 0.8815
Epoch 829/1000
2s - loss: 0.2523 - acc: 0.9028 - val_loss: 0.3014 - val_acc: 0.8795
Epoch 830/1000
2s - loss: 0.2528 - acc: 0.8999 - val_loss: 0.3046 - val_acc: 0.8805
Epoch 831/1000
2s - loss: 0.2530 - acc: 0.9026 - val_loss: 0.3029 - val_acc: 0.8820
Epoch 832/1000
2s - loss: 0.2523 - acc: 0.9030 - val_loss: 0.3002 - val_acc: 0.8830
Epoch 833/1000
1s - loss: 0.2512 - acc: 0.9019 - val_loss: 0.3111 - val_acc: 0.8710
Epoch 834/1000
2s - loss: 0.2527 - acc: 0.9008 - val_loss: 0.3046 - val_acc: 0.8755
Epoch 835/1000
1s - loss: 0.2524 - acc: 0.9035 - val_loss: 0.3012 - val_acc: 0.8775
Epoch 836/1000
2s - loss: 0.2523 - acc: 0.9017 - val_loss: 0.3032 - val_acc: 0.8790
Epoch 837/1000
2s - loss: 0.2514 - acc: 0.9033 - val_loss: 0.3124 - val_acc: 0.8710
Epoch 8

2s - loss: 0.2458 - acc: 0.9042 - val_loss: 0.2967 - val_acc: 0.8800
Epoch 925/1000
2s - loss: 0.2457 - acc: 0.9053 - val_loss: 0.2995 - val_acc: 0.8765
Epoch 926/1000
2s - loss: 0.2441 - acc: 0.9069 - val_loss: 0.3053 - val_acc: 0.8670
Epoch 927/1000
3s - loss: 0.2448 - acc: 0.9076 - val_loss: 0.2970 - val_acc: 0.8750
Epoch 928/1000
3s - loss: 0.2449 - acc: 0.9033 - val_loss: 0.2961 - val_acc: 0.8780
Epoch 929/1000
2s - loss: 0.2453 - acc: 0.9051 - val_loss: 0.2969 - val_acc: 0.8820
Epoch 930/1000
2s - loss: 0.2453 - acc: 0.9057 - val_loss: 0.2986 - val_acc: 0.8705
Epoch 931/1000
2s - loss: 0.2444 - acc: 0.9069 - val_loss: 0.2963 - val_acc: 0.8795
Epoch 932/1000
2s - loss: 0.2445 - acc: 0.9019 - val_loss: 0.2931 - val_acc: 0.8845
Epoch 933/1000
2s - loss: 0.2449 - acc: 0.9048 - val_loss: 0.2956 - val_acc: 0.8830
Epoch 934/1000
2s - loss: 0.2448 - acc: 0.9062 - val_loss: 0.2997 - val_acc: 0.8750
Epoch 935/1000
2s - loss: 0.2451 - acc: 0.9030 - val_loss: 0.2951 - val_acc: 0.8790
Epoch 9

KeyboardInterrupt: 

In [ ]:
# Question 5
# design a 4-layer network with two hidden- layers, 
# each consisting of 10 neurons with logistic activation functions, 
# batch size of 32 and decay parameter 10-6.


learning_rate = 0.01
epoch_num = 1000
decay = math.pow(10,-6)
batch_time_taken = []
total_time_taken = []

bt = BatchTime()
start_time = time.time()
inputs = Input(shape=(trainX.shape[1],))
dense1 = Dense(10, activation='sigmoid')(inputs)
dense2 = Dense(10, activation='sigmoid')(dense1)
predictions = Dense(trainY.shape[1], activation='softmax')(dense2)

model = Model(inputs=inputs, outputs=predictions)
sgd = optimizers.SGD(lr=learning_rate, decay=decay)
model.compile(optimizer=sgd,
              loss='categorical_crossentropy',
              metrics=['accuracy'])
start_time = time.time()
train_his = model.fit(trainX, trainY, epochs=epoch_num, verbose=2, 
                      validation_data=[testX,testY],
                      callbacks=[bt],
                      batch_size = 32)

print("Time taken to train a 4-layer network is : " + str(time.time() - start_time))
    
#  Plot the train and test accuracy of the 4-layer network.
# Plot the training error and the test accuracy against number of epochs
plt.figure()
plt.plot(range(1, 1+epoch_num), train_his.history['loss'], label='train_cost')
plt.xlabel('iterations')
plt.ylabel('cross-entropy')
plt.title('training cost')
annot_min(range(1, 1+epoch_num), np.array(train_his.history['loss']))
plt.savefig(plot_pic_folder + '5/p1a_cost.png')
plt.gcf().clear()
plt.close()


plt.figure()
plt.plot(range(1, 1+epoch_num), train_his.history['val_acc'], label='test_accuracy')
plt.xlabel('iterations')
plt.ylabel('accuracy')
plt.title('test accuracy')
annot_max(range(1, 1+epoch_num), np.array(train_his.history['val_acc']))
plt.savefig(plot_pic_folder + '5/p1a_sample_accuracy.png')
plt.close()
plt.gcf().clear()
plt.close()
# b) Compare and comment on the performances on 3-layer and 4-layer networks.